In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [74]:
berdo = pd.read_csv(".\\data\\berdo.csv")

print(berdo.columns)

cols_num = ['Gross Area (sq ft)', 'Site EUI (kBTU/sf)',
        'GHG Emissions (MTCO2e)', 'GHG Intensity (kgCO2/sf)',
        ' Total Site Energy (kBTU) ','Water Intensity (gal/sf)']

cols_percent = ["% Gas", "% Steam", "% Electricity"]

def to_number(s):
    try:
        float(s)
        return float(s)
    except ValueError:
        return -1

def fix_numerical(columns):
    
    for c in columns:
        for year in ("_2014","_2015"):
            cyear = c + year
            berdo[cyear] = berdo[cyear].map(lambda x: str(x).strip().replace(",",""))
            berdo[cyear] = berdo[cyear].map(to_number)
            cyear = c
            
def fix_percenteage(columns):
    
    for c in columns:
        for year in ("_2014","_2015"):
            cyear = c + year
            berdo[cyear].fillna("0%", inplace=True)
            berdo[cyear] = berdo[cyear].map(lambda x: float(x.strip().replace("%","")))         
            cyear = c

fix_numerical(cols_num)
fix_percenteage(cols_percent)

berdo["%_total_2015"] = berdo[["% Gas_2015","% Steam_2015","% Electricity_2015"]].sum(axis=1)
berdo["%_total_2014"] = berdo[["% Gas_2014","% Steam_2014","% Electricity_2014"]].sum(axis=1)

Index(['Property Name', 'Reported_2015', 'Property Type_2015', 'Address',
       'ZIP_2015', 'Gross Area (sq ft)_2015', 'Site EUI (kBTU/sf)_2015',
       'Energy Star Score_2015', 'Energy Star Certified_2015',
       'Property Uses_2015', 'Year Built_2015', 'GHG Emissions (MTCO2e)_2015',
       'GHG Intensity (kgCO2/sf)_2015', ' Total Site Energy (kBTU) _2015',
       '% Electricity_2015', '% Gas_2015', '% Steam_2015',
       'Water Intensity (gal/sf)_2015', 'Onsite Solar (kWh)_2015',
       'User Submitted Info_2015', 'User Submitted Link_2015',
       'Tax Parcel_2015', 'Years Reported', 'Reported_2014',
       'Property Type_2014', 'ZIP_2014', 'Gross Area (sq ft)_2014',
       'Site EUI (kBTU/sf)_2014', 'Energy Star Score_2014',
       'Energy Star Certified_2014', 'Property Uses_2014', 'Year Built_2014',
       'GHG Emissions (MTCO2e)_2014', 'GHG Intensity (kgCO2/sf)_2014',
       ' Total Site Energy (kBTU) _2014', '% Electricity_2014', '% Gas_2014',
       '% Steam_2014', 'Water I

In [75]:
berdo = berdo[["Property Name","Address","ZIP_2015","Year Built_2015",
       "Property Type_2015","Property Type_2014",
       "Gross Area (sq ft)_2015","Gross Area (sq ft)_2014",
       "Site EUI (kBTU/sf)_2015","Site EUI (kBTU/sf)_2014",
       " Total Site Energy (kBTU) _2015"," Total Site Energy (kBTU) _2014",
       "GHG Emissions (MTCO2e)_2015","GHG Emissions (MTCO2e)_2014",
       "GHG Intensity (kgCO2/sf)_2015","GHG Intensity (kgCO2/sf)_2014",
       "% Electricity_2015", "% Gas_2015", "% Steam_2015","%_total_2015",
       "% Electricity_2014", "% Gas_2014", "% Steam_2014","%_total_2014"
      ]]

### remove data that doesn't seem right. A deeper investigation should be done here
#print(berdo.isnull().sum())
print(berdo.shape)

print("---------")

# set values -1 to NaN
berdo.replace(-1,np.NaN,inplace=True)
# remove if total energy < 90. Check these deeper
berdo = berdo[(berdo["%_total_2015"] > 90) & (berdo["%_total_2014"] > 90)]
# drop all missing values
berdo.dropna(axis=0, how='any', inplace=True)
print(berdo.shape)

Property Name                      0
Address                            0
ZIP_2015                           0
Year Built_2015                    0
Property Type_2015                 0
Property Type_2014                 0
Gross Area (sq ft)_2015            0
Gross Area (sq ft)_2014            0
Site EUI (kBTU/sf)_2015            0
Site EUI (kBTU/sf)_2014            0
 Total Site Energy (kBTU) _2015    0
 Total Site Energy (kBTU) _2014    0
GHG Emissions (MTCO2e)_2015        0
GHG Emissions (MTCO2e)_2014        0
GHG Intensity (kgCO2/sf)_2015      0
GHG Intensity (kgCO2/sf)_2014      0
% Electricity_2015                 0
% Gas_2015                         0
% Steam_2015                       0
%_total_2015                       0
% Electricity_2014                 0
% Gas_2014                         0
% Steam_2014                       0
%_total_2014                       0
dtype: int64
(1086, 24)
---------
Property Name                      0
Address                            0
ZIP_

In [90]:
berdo_diff = pd.DataFrame()
berdo_diff[["property_name", "address","zip","year_built"]] = berdo[["Property Name","Address","ZIP_2015","Year Built_2015"]]
berdo_diff[["property_type_2015","property_type_2014"]] = berdo[["Property Type_2015","Property Type_2014"]]
berdo_diff["gloss_area_sqft"] = berdo["Gross Area (sq ft)_2015"]
berdo_diff["gloss_area_sqft_diff"] = berdo["Gross Area (sq ft)_2015"] - berdo["Gross Area (sq ft)_2014"]
berdo_diff["energy_kbtu/sf_diff"] = berdo["Site EUI (kBTU/sf)_2015"] - berdo["Site EUI (kBTU/sf)_2014"]
berdo_diff["energy_total_kbtu_diff"] = berdo[" Total Site Energy (kBTU) _2015"] - berdo[" Total Site Energy (kBTU) _2014"]
berdo_diff["ghg_emissions_mtco2e_diff"] = berdo["GHG Emissions (MTCO2e)_2015"] - berdo["GHG Emissions (MTCO2e)_2014"]
berdo_diff["ghg_intensity_kgco2/sf_diff"] = berdo["GHG Intensity (kgCO2/sf)_2015"] - berdo["GHG Intensity (kgCO2/sf)_2014"]

In [91]:
berdo_diff.head()

,property_name,address,zip,year_built,property_type_2015,property_type_2014,gloss_area_sqft,gloss_area_sqft_diff,energy_kbtu/sf_diff,energy_total_kbtu_diff,ghg_emissions_mtco2e_diff,ghg_intensity_kgco2/sf_diff
0,#2679 south bay/boston,5 alllstate road,02125,1998,Retail Store,Retail Store,132000.0,264000.0,6.9,912943.0,47.5,0.4
1,0004 roslindale,950 american legion hgwy,02131,2001,Supermarket/Grocery Store,Supermarket/Grocery Store,38694.0,77388.0,-2.8,-107313.0,-19.4,-0.5
2,0018 dorchester,545 freeport street,02122,2005,Supermarket/Grocery Store,Supermarket/Grocery Store,61548.0,123096.0,-2.3,-146136.1,-31.9,-0.5
3,0053 south boston,713 e. broadway,02127,2005,Supermarket/Grocery Store,Supermarket/Grocery Store,30982.0,61964.0,-4.9,-152937.9,-2.6,0.0
4,0089 allston,60 everett st.,02134,2005,Supermarket/Grocery Store,Supermarket/Grocery Store,79894.0,159788.0,-15.9,-1275106.8,-100.9,-1.2


In [80]:
berdo_diff["gloss_area_sqft_diff"].value_counts()

 0.0         770
-200000.0      2
-18000.0       2
-23242.0       1
-2500.0        1
-56998.0       1
-35500.0       1
-50863.0       1
-86715.0       1
-419328.0      1
 13620.0       1
-211604.0      1
-234189.0      1
-298708.0      1
-24278.0       1
-311000.0      1
-140000.0      1
-225000.0      1
-15000.0       1
 1400.0        1
-14064.0       1
-270840.0      1
-235000.0      1
-250368.0      1
-91515.0       1
-21000.0       1
-70161.0       1
-548.0         1
-190000.0      1
-34372.0       1
            ... 
-75180.0       1
-118195.0      1
 61000.0       1
 221764.0      1
-112068.0      1
 561.0         1
-421653.0      1
-13784.0       1
-407000.0      1
-1500.0        1
 2674.0        1
-88458.0       1
-7475.0        1
-138633.0      1
-310580.0      1
-33088.0       1
-68012.0       1
-269637.0      1
-70983.0       1
-115019.0      1
-55024.0       1
-91471.0       1
 66214.0       1
-21858.0       1
-73059.0       1
-30075.0       1
-48510.0       1
-3456.0       

In [19]:
set(berdo["Property Type_2015"].value_counts().index.tolist()) - set(berdo["Property Type_2014"].value_counts().index.tolist())

{'Other - Services'}

In [21]:
berdo.shape

(1086, 43)

In [20]:
berdo.isnull().sum()

Property Name                         0
Reported_2015                         0
Property Type_2015                    0
Address                               0
ZIP_2015                              0
Gross Area (sq ft)_2015               0
Site EUI (kBTU/sf)_2015               0
Energy Star Score_2015                0
Energy Star Certified_2015         1019
Property Uses_2015                    7
Year Built_2015                       0
GHG Emissions (MTCO2e)_2015           0
GHG Intensity (kgCO2/sf)_2015         0
 Total Site Energy (kBTU) _2015       0
% Electricity_2015                   46
% Gas_2015                          215
% Steam_2015                        986
Water Intensity (gal/sf)_2015       336
Onsite Solar (kWh)_2015            1081
User Submitted Info_2015            893
User Submitted Link_2015           1086
Tax Parcel_2015                       0
Years Reported                        0
Reported_2014                         0
Property Type_2014                    0


In [44]:
berdo[["% Gas_2015","% Steam_2015","% Electricity_2015"]].sum(axis=1).value_counts().loc[50:]

50.0    1
94.0    1
98.0    1
61.0    1
64.0    1
67.0    1
84.0    1
87.0    1
92.0    1
93.0    1
59.0    1
dtype: int64

In [51]:
berdo[["% Gas_2014","% Steam_2014","% Electricity_2014"]].sum(axis=1).value_counts()

100.0    948
0.0       69
99.0       7
38.0       3
67.0       3
71.0       3
35.0       2
57.0       2
60.0       2
32.0       2
72.0       2
74.0       2
76.0       2
12.0       2
5.0        2
26.0       1
24.0       1
28.0       1
23.0       1
31.0       1
18.0       1
17.0       1
13.0       1
33.0       1
11.0       1
10.0       1
7.0        1
34.0       1
37.0       1
101.0      1
42.0       1
44.0       1
98.0       1
97.0       1
93.0       1
92.0       1
90.0       1
89.0       1
88.0       1
82.0       1
79.0       1
78.0       1
77.0       1
75.0       1
73.0       1
69.0       1
66.0       1
48.0       1
45.0       1
56.0       1
dtype: int64

In [48]:
berdo["%_total_2015"] = berdo[["% Gas_2015","% Steam_2015","% Electricity_2015"]].sum(axis=1)
berdo["%_total_2014"] = berdo[["% Gas_2014","% Steam_2014","% Electricity_2014"]].sum(axis=1)

0       100.0
1       100.0
2       100.0
3       100.0
4       100.0
5       100.0
6       100.0
7       100.0
8       100.0
9       100.0
10      100.0
11      100.0
12      100.0
13      100.0
14      100.0
15      100.0
16      100.0
17       99.0
18      100.0
19      100.0
20      100.0
21      100.0
22      100.0
23      100.0
24      100.0
25      100.0
26      100.0
27      100.0
28      100.0
29      100.0
        ...  
1056    100.0
1057    100.0
1058    100.0
1059    100.0
1060    100.0
1061    100.0
1062    100.0
1063    100.0
1064    100.0
1065    100.0
1066    100.0
1067    100.0
1068    100.0
1069     51.0
1070    100.0
1071    100.0
1072    100.0
1073    100.0
1074    100.0
1075      0.0
1076    100.0
1077    100.0
1078    100.0
1079    100.0
1080    100.0
1081    100.0
1082    100.0
1083    100.0
1084    100.0
1085      0.0
dtype: float64

In [31]:
berdo[berdo["Year Built_2015"] != berdo["Year Built_2014"]][["Year Built_2015","Year Built_2014"]]

,Year Built_2015,Year Built_2014
197,2010,2008.0
793,2013,2000.0
887,1968,1978.0
893,2010,2012.0
975,1984,1899.0


In [ ]:
berdo[["Property Name","Address","ZIP_2015","Year Built_2015"
       "Property Type_2015","Property Type_2014"
       "Gross Area (sq ft)_2015","Gross Area (sq ft)_2014",
       "Site EUI (kBTU/sf)_2015","Site EUI (kBTU/sf)_2014",
       " Total Site Energy (kBTU) _2015"," Total Site Energy (kBTU) _2014",
       "GHG Emissions (MTCO2e)_2015","GHG Emissions (MTCO2e)_2014",
       "GHG Intensity (kgCO2/sf)_2015","GHG Intensity (kgCO2/sf)_2014"
      ]]